In [1]:
# Load all packages
from os.path import join
import mat73
import matplotlib.pyplot as plt
import numpy as np
import mne
from tqdm import tqdm # For progress bar

In [2]:
# Load data into dictionary
def LoadData(subjectpath,rootdir="neuro_data"):
    """Loads .mat data into dictionary

    Args:
        subjectfile (str): filename of subject .mat file
        rootdir (str, optional): path to directory holding subject data. Defaults to "neuro_data".

    Returns:
        dict: dictionary with subject data
    """
    DataPath = join(rootdir,subjectpath)
    data_dict = mat73.loadmat(DataPath, use_attrdict=True)
    return data_dict["data"]

In [4]:
# Function to transform data to appropriate shape

def DimConvertFunc(trial, all_channels, bad_channels, n_samples):
    """Converts 60 channel data to 74 channel data with dummy info, in order to fit EasyCap format.

    Args:
        trial (array of floats): data from channel
        moving_window (int, optional): moving average window to smoothen data for visualization. Defaults to 0.

    Returns:
        _type_: (n_trials, n_samples)
    """
    newTrial = np.zeros((len(all_channels),n_samples))
    j = 0
    filler = [0]*n_samples
    for i, ch in enumerate(all_channels):
        if ch in bad_channels:
            newTrial[i]=filler
        else:
            newTrial[i]=trial[j]/(10**6) # Divide with ~10**6 to convert muV to V
            j += 1
    return newTrial

In [5]:
def create_MNE_events(trialinfo):
    """Create event data structure with MNE compatible format from data["trialinfo"]

    Args:
        trialinfo (_type_): data["trialinfo"] as provided in the MatLab data from M. Sandsten.

    Returns:
        _type_: MNE compatible event data structure
    """

    n = len(trialinfo)
    metadata = [0]*n
    for i, trialinfo in enumerate(trialinfo):
        # side : left = 1, right = 2
        # side_accuracy : Correct (1) Incorrect (2) No response (3) Almost correct (4)
        metadata[i] = {"side":int(trialinfo[0]["side"]),"accurate":int(trialinfo[0]["side_accuracy"])}

    event_label = [event["side"] for event in metadata] # actual info, must be converted to shape (N,3)
    events = np.zeros((n,3))
    events[:,2] = np.array(event_label)  #Add labels to 3rd column, 2nd column irrelevant, but shape needed
    events[:,0] = np.array(range(n))     #Add dummy IDs to first column
    events = events.astype(int)
    return events

In [12]:
def create_MNE_EpochsArray(data):
    # Constants
    sfreq = data["fsample"] # Sampling frequency
    used_channels = [ch[0].upper() for ch in data["label"]]
    all_channels = [ch[0] for ch in data["elec"]["label"]] # Not all upper case
    used_channel_case = [ch for ch in all_channels if ch.upper() in used_channels] # match cases
    bad_channels = list(set(all_channels).difference(set(used_channel_case))) # Channels not used
    n_samples = len(data["trial"][0][0]) # Amount of measurements for each channel in a trial

    # MNE data structures

    info = mne.create_info(all_channels, sfreq, "eeg")
    info.set_montage('easycap-M1')
    info["bads"] = bad_channels

    events = create_MNE_events(data["trialinfo"])
    event_dict = {"auditory/left":1, "auditory/right":2}

    # Create array with 74 instead of 60 channel data to fit Easycap Scheme
    rawdata_UpScaledDim = np.array([DimConvertFunc(epoch, all_channels, bad_channels, n_samples) for epoch in data["trial"]])
    epochs = mne.EpochsArray(rawdata_UpScaledDim, info, tmin=-2, events=events, event_id=event_dict, baseline=(-2,0))
    infodict = {"epochs":epochs, 
                "used_channels":used_channel_case, 
                "bad_channels":bad_channels, 
                "events":events, 
                "event_dict":event_dict}
    return infodict

In [29]:
def saveEpoFIF(subject, destination):
    """Takes .mat eeg file provided by M. Sandsten and converts to MNE epoch .fif file

    Args:
        subject (string): name of subject .mat file
        destination (string): location to save -epo.fif file
    """
    data = LoadData(subject)
    epodict = create_MNE_EpochsArray(data)
    filename = subject.split(".")[0]+"-epo.fif"
    DataPath = join(destination,filename)
    epodict["epochs"].save(DataPath)

In [34]:
destination = join("neuro_data","raw_epofif")
saveEpoFIF("dataSubj15.mat",destination)

Not setting metadata
370 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
